This requests a list of known charging stations

In [17]:
import requests
from geopy.distance import geodesic
import json

def parsecoordinates(a):
    if isinstance(a, bytes):
        a = a.decode('utf-8')
    
    # Entferne unerwünschte Leerzeichen oder Sonderzeichen
    a = a.strip()

    latitude, longitude = map(float, a.split(','))
    coordinate = (latitude, longitude)
    return coordinate

homerequest=requests.get('https://ipinfo.io/loc')
if(homerequest.status_code!=200):
    print('cannot get home location')
    exit

home = parsecoordinates(homerequest.content)
print('home: ', home)


csrequest=requests.get('https://appbackend.chargeatfriends.com/api/search')
if(csrequest.status_code!=200):
    print('network error')
    exit

cs=csrequest.json();
csitems=[]
if isinstance(cs, list):  # Prüft, ob die Antwort ein Array ist
    for csitem in cs:
        # Prüfen, ob die Schlüssel vorhanden sind
        if 'Lat' in csitem and 'Lon' in csitem:
            location = f"{csitem['Lat']},{csitem['Lon']}"
            location=parsecoordinates(location)
            csitem["Location"]=location
            csitem["Distance"] = round((geodesic(home, location).kilometers),1)
            #print(csitem)
            #print(csitem["distance"],'KM')

            if csitem["Distance"] <= 50:
                csitems.append(csitem)
                #print("adding ", csitem)
            #else:
                #print("ignore ", csitem)
        else:
            print("Lat oder Lon fehlt in einem Objekt")
else:
    print("Die JSON-Daten sind kein Array.")

# Sortiere nach dem Attribut 'Distance'
csitems = sorted(csitems, key=lambda x: x['Distance'])
for csitem in csitems:
    print(csitem)
    print(csitem["Distance"],"KM ,",csitem["CpName"],":",csitem["CurrentState"],", Tarif:",csitem["Tarif"],", Parken Tag:",csitem["SunShineParkingRate"],", Parken Nacht:",csitem["MoonShineParkingRate"])


home:  (51.4322, 6.8797)
{'Guid': '29310113-a8b5-498e-98d4-5c31afe38581', 'RowKey': '29310113-a8b5-498e-98d4-5c31afe38581', 'PartitionKey': '254ec41c-2ec4-80fb-0a89-a30088c29e84', 'ConnectorId': 1, 'EvseId': None, 'OwnerGuid': '254ec41c-2ec4-80fb-0a89-a30088c29e84', 'Name': 'Grün', 'CpName': 'Einfahrt', 'Owner': 'jonas@plitt.de', 'OwnerFullName': None, 'UStNr': None, 'Lat': 51.42537351216869, 'Lon': 6.872660953375567, 'AutoApprovalTimeout': 180, 'OperatorId': 0, 'Type': 2196, 'Connector': 0, 'State': 96, 'CurrentState': 'free', 'Available': True, 'IsAvailable': True, 'PriceCalcGuid': '9b6adbf0-03aa-4e3f-ad4e-41a26c164b21', 'Tarif': 32.0, 'ServiceFee': 0.0, 'MinimalFee': 0, 'SunShineRate': 32.0, 'MoonShineRate': 32.0, 'SunShineParkingRate': 0.0, 'MoonShineParkingRate': 0.0, 'Power': 22.0, 'ThumbnailUrl': 'https://cafproduction.blob.core.windows.net/chargepoint/9ddce3eb-20fe-4860-b483-ef15050b3990/9ddce3eb-20fe-4860-b483-ef15050b3990.jpg?221', 'Street': 'Am Bahnhof Broich', 'StreetNumber